Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
В качестве решения: Jupyter notebook с кодом, комментариями и графиками

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import auc, roc_curve, roc_auc_score


In [2]:
train = pd.read_csv('../data/HousePrices/train.csv')
test = pd.read_csv('../data/HousePrices/test.csv')
test_Y = pd.read_csv('../data/HousePrices/sample_submission.csv')


### 1 Информация о данных

In [3]:
print(train.shape,test.shape,test_Y.shape)

(1460, 81) (1459, 80) (1459, 2)


In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [6]:
#train.info()
tp = train.isnull().sum()>0
tp[tp]
# что-то не придумал более изящного кода

LotFrontage     True
Alley           True
MasVnrType      True
MasVnrArea      True
BsmtQual        True
BsmtCond        True
BsmtExposure    True
BsmtFinType1    True
BsmtFinType2    True
Electrical      True
FireplaceQu     True
GarageType      True
GarageYrBlt     True
GarageFinish    True
GarageQual      True
GarageCond      True
PoolQC          True
Fence           True
MiscFeature     True
dtype: bool

In [7]:
numeric=[f for f in train.columns if train.dtypes[f] != 'object']
num_features = train.select_dtypes(include=[np.number])

catagorical = [i for i in train.columns if train.dtypes[i] == 'object']
cat_features = train.select_dtypes(include=[object])
print(numeric)
print(catagorical)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'Gara

#### В данных 80 признаков. 38 числовых. 19 признаков имеют пропущенные значения.

### Удаляем незначемые признаки и выделяем Target

In [8]:
train_Y = np.log1p(train.SalePrice.values)
#back = np.expm1(Y)

train_X = train.drop(['Id','SalePrice'], axis=1)
test_X = test.drop(['Id'], axis=1)

print(train_X.shape) 
print(test_X.shape)

ntrain = train_X.shape[0]
ntest = test_X.shape[0]
all_data = pd.concat((train_X, test_X)).reset_index(drop=True)

#print(train_X.head(5)) 
#print(test_X.head(5))
#print(all_data.head(5))


print("train : {}, train_X : {}, all_data : {} ".format(train.shape,train_X.shape,all_data.shape))

(1460, 79)
(1459, 79)
train : (1460, 81), train_X : (1460, 79), all_data : (2919, 79) 


### 2 Подготовка данных

### 2.1 Заполняем пропуски

In [9]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
# заполняем недостающие значения медианной площадью в этом районе
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data["Utilities"] = all_data["Utilities"].fillna("None")
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")


In [10]:
tp = all_data.isnull().sum()>0
tp[tp]

Series([], dtype: bool)

#### В данных пустот не осталось

### 2.2 Преобразуем категориальные признаки в числовые

In [11]:
# Некоторые признаки хоть и имеют числовой тип, но реально их природа категориальная
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [12]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2919, 79)


In [13]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2919, 223)


In [14]:
train_XX = all_data[:ntrain]
test_XX = all_data[ntrain:]
print('train_XX: {}, test_XX: {}'.format(train_XX.shape,test_XX.shape))

train_XX: (1460, 223), test_XX: (1459, 223)


### 3 Модель RandomForest

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
model_1 = RandomForestRegressor(n_estimators=100, n_jobs=2)
model_1.fit(train_XX,train_Y)
pred_Y = np.expm1(model_1.predict(test_XX))
print(model_1.score(test_XX,np.log1p(test_Y['SalePrice'])))


-15.350488563242699


In [17]:
# Фичи отсортированные от важных к бесполезным
imp_f = np.sort(model_1.feature_importances_)
print(imp_f[::-1])

[5.42579163e-01 1.11358336e-01 4.66442923e-02 4.39573717e-02
 2.38502660e-02 2.11627138e-02 2.05865295e-02 1.70359215e-02
 1.23409642e-02 1.20485498e-02 1.18872944e-02 8.70152318e-03
 8.34313703e-03 6.97715438e-03 6.71522760e-03 6.67706125e-03
 5.58464342e-03 3.60999281e-03 3.55259691e-03 3.47910065e-03
 3.11318291e-03 3.03169448e-03 3.02425214e-03 2.94657239e-03
 2.91759359e-03 2.77091471e-03 2.69048568e-03 2.46930049e-03
 2.31606415e-03 2.16861367e-03 2.14564223e-03 2.12198981e-03
 1.92088074e-03 1.84267573e-03 1.69598639e-03 1.67593541e-03
 1.64512918e-03 1.53960968e-03 1.47722158e-03 1.35765613e-03
 1.31818069e-03 1.17298742e-03 1.11979312e-03 1.08783127e-03
 1.02670697e-03 9.84806459e-04 9.80443696e-04 9.41385272e-04
 9.22759193e-04 8.23007192e-04 7.67771109e-04 7.57385266e-04
 7.51117725e-04 7.48319907e-04 7.45481314e-04 7.37764549e-04
 6.83514226e-04 6.60775633e-04 6.50703937e-04 6.44756821e-04
 6.41392501e-04 5.80624948e-04 5.51375306e-04 5.12060829e-04
 4.88660820e-04 4.751090

### 4. Обучаем стекинг 3х моделей ( LinearRegression, KNeighborsRegressor, DecisionTreeRegressor )

In [20]:
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [21]:
estimators = [
                ('lr',   LinearRegression()),
                ('knr',  KNeighborsRegressor(n_neighbors=3)),
                ('tree', DecisionTreeRegressor())]

reg = StackingRegressor(estimators=estimators,
                        final_estimator=RandomForestRegressor(n_estimators=10))

reg.fit(train_XX,train_Y).score(test_XX,np.log1p(test_Y['SalePrice']))

-16.88493491348553